### **Data Pre-Processing**

In [513]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk
import textblob

from textblob import TextBlob
from textblob import Word
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer

In [476]:
df_raw = pd.read_excel("amazon_review.xlsx")
df_raw.head(5)

,Review Model,Retailer,Review date,Review name,Review rating,Review title,Review Content,Verified Purchase or not,People_find_helpful,vine or not,URL,list price,rating count,overall rating
0,Canon Pixma TS6420a,Amazon,2023-08-10,Ernest Birkholz,5,Works great,🖨 was easy to install and works great.,Verified Purchase,NaN,NaN,https://www.amazon.com/product-reviews/B09TG52...,129.99,285,4.2
1,HP OfficeJet Pro 9015e,Amazon,2022-06-04,mattey,3,spunky mid size printer,🔆Slower print speed than what I’m used to (old...,Verified Purchase,NaN,NaN,https://www.amazon.com/product-reviews/B08QR6P...,289.99,285,4.2
2,Canon PIXMA MG3620,Amazon,2023-03-15,Maria D,4,𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜,𝙻𝚘𝚟𝚎 𝚒𝚝,Verified Purchase,NaN,NaN,https://www.amazon.com/Canon-MG3620-Wireless-P...,79.99,285,4.2
3,Epson - ET-3830,Amazon,2022-11-17,Ryan H,5,Shaq knows what he's talking about,"Yup, this printer is a slam dunk. :)What an up...",Verified Purchase,12.0,NaN,https://www.amazon.com/product-reviews/B096NBP...,399.99,285,4.2
4,HP ENVY 6055e,Amazon,2022-04-09,Sam,3,Not User Friendly,You would think something as simple as reconne...,Verified Purchase,NaN,NaN,https://www.amazon.com/product-reviews/B08XYRV...,129.99,285,4.2


In [477]:
df_raw['vine or not'].fillna('', inplace=True)
df_raw['People_find_helpful'].fillna('', inplace=True)
df_raw['Verified Purchase or not'].fillna('', inplace=True)

In [478]:
df_raw['vine or not'] = df_raw['vine or not'].replace('', 'Not VINE VOICE')
df_raw['People_find_helpful'] = df_raw['People_find_helpful'].replace('', 0)
df_raw['Verified Purchase or not'] = df_raw['Verified Purchase or not'].replace('', 'Not Verified')

In [479]:
df_raw['Review Model'] = df_raw['Review Model'].astype(str)
df_raw['Retailer'] = df_raw['Retailer'].astype(str)
df_raw['Review title'] = df_raw['Review title'].astype(str)
df_raw['Review Content'] = df_raw['Review Content'].astype(str)
df_raw['Verified Purchase or not'] = df_raw['Verified Purchase or not'].astype(str)
df_raw['People_find_helpful'] = df_raw['People_find_helpful'].astype(str)

#### <font color=#FFB703> **Checking for Missing Values** </font>

In [480]:
missing_values = df_raw.isnull().sum()
missing_values

Review Model                 0
Retailer                     0
Review date                  0
Review name                 57
Review rating                0
Review title                 0
Review Content               0
Verified Purchase or not     0
People_find_helpful          0
vine or not                  0
URL                          0
list price                   0
rating count                 0
overall rating               0
dtype: int64

#### <font color=#FFB703> **Summarising Review Rating** </font>

In [481]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10209 entries, 0 to 10208
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Review Model              10209 non-null  object        
 1   Retailer                  10209 non-null  object        
 2   Review date               10209 non-null  datetime64[ns]
 3   Review name               10152 non-null  object        
 4   Review rating             10209 non-null  int64         
 5   Review title              10209 non-null  object        
 6   Review Content            10209 non-null  object        
 7   Verified Purchase or not  10209 non-null  object        
 8   People_find_helpful       10209 non-null  object        
 9   vine or not               10209 non-null  object        
 10  URL                       10209 non-null  object        
 11  list price                10209 non-null  float64       
 12  rating count      

#### <font color=#FFB703> **Feature Extraction** </font>

<font color=#61BEB0>**Number of Words:**</font>

In [482]:
# Number of words (Review Content)
df_raw['content_word_count'] = df_raw['Review Content'].apply(lambda x: len(str(x).split(" ")))
df_raw[['Review Content','content_word_count']].head()

,Review Content,content_word_count
0,🖨 was easy to install and works great.,8
1,🔆Slower print speed than what I’m used to (old...,79
2,𝙻𝚘𝚟𝚎 𝚒𝚝,2
3,"Yup, this printer is a slam dunk. :)What an up...",84
4,You would think something as simple as reconne...,160


In [483]:
# Number of words (Review title)
df_raw['title_word_count'] = df_raw['Review title'].apply(lambda x: len(str(x).split(" ")))
df_raw[['Review title','title_word_count']].head()

,Review title,title_word_count
0,Works great,2
1,spunky mid size printer,4
2,𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜,2
3,Shaq knows what he's talking about,6
4,Not User Friendly,3


<font color=#61BEB0>**Number of Characters:**</font>

In [484]:
# Number of characters (Review Content)
df_raw['content_char_count'] = df_raw['Review Content'].str.len() ## this also includes spaces
df_raw[['Review Content','content_char_count']].head()

,Review Content,content_char_count
0,🖨 was easy to install and works great.,38
1,🔆Slower print speed than what I’m used to (old...,460
2,𝙻𝚘𝚟𝚎 𝚒𝚝,7
3,"Yup, this printer is a slam dunk. :)What an up...",469
4,You would think something as simple as reconne...,861


In [485]:
# Number of characters (Review title)
df_raw['title_char_count'] = df_raw['Review title'].str.len() ## this also includes spaces
df_raw[['Review title','title_char_count']].head()

,Review title,title_char_count
0,Works great,11
1,spunky mid size printer,23
2,𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜,12
3,Shaq knows what he's talking about,34
4,Not User Friendly,17


<font color=#61BEB0>**Average Word Length:**</font>

In [486]:
# Average word length (Review Content)
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

df_raw['content_avg_word'] = df_raw['Review Content'].apply(lambda x: avg_word(x))
df_raw[['Review Content','content_avg_word']].head()

,Review Content,content_avg_word
0,🖨 was easy to install and works great.,3.875000
1,🔆Slower print speed than what I’m used to (old...,4.835443
2,𝙻𝚘𝚟𝚎 𝚒𝚝,3.000000
3,"Yup, this printer is a slam dunk. :)What an up...",4.595238
4,You would think something as simple as reconne...,4.387500


In [487]:
# Average word length (Review title)
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

df_raw['title_avg_word'] = df_raw['Review title'].apply(lambda x: avg_word(x))
df_raw[['Review title','title_avg_word']].head()

,Review title,title_avg_word
0,Works great,5.000000
1,spunky mid size printer,5.000000
2,𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜,5.500000
3,Shaq knows what he's talking about,4.833333
4,Not User Friendly,5.000000


<font color=#61BEB0>**Number of Stop Words:**</font>

In [488]:
# Number of stop words (Review Content)
stop = stopwords.words('english')

df_raw['content_stopwords'] = df_raw['Review Content'].apply(lambda x: len([x for x in x.split() if x in stop]))
df_raw[['Review Content','content_stopwords']].head()

,Review Content,content_stopwords
0,🖨 was easy to install and works great.,3
1,🔆Slower print speed than what I’m used to (old...,19
2,𝙻𝚘𝚟𝚎 𝚒𝚝,0
3,"Yup, this printer is a slam dunk. :)What an up...",31
4,You would think something as simple as reconne...,71


In [489]:
# Number of stop words (Review title)
stop = stopwords.words('english')

df_raw['title_stopwords'] = df_raw['Review title'].apply(lambda x: len([x for x in x.split() if x in stop]))
df_raw[['Review title','title_stopwords']].head()

,Review title,title_stopwords
0,Works great,0
1,spunky mid size printer,0
2,𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜,0
3,Shaq knows what he's talking about,2
4,Not User Friendly,0


#### <font color=#FFB703> **Text Pre-Processing** </font>

<font color=#61BEB0>**Brand Extraction:**</font>

In [490]:
df_raw['Brand'] = df_raw['Review Model'].str.split().str[0]
df_raw[['Brand', 'Review Model']].head()

,Brand,Review Model
0,Canon,Canon Pixma TS6420a
1,HP,HP OfficeJet Pro 9015e
2,Canon,Canon PIXMA MG3620
3,Epson,Epson - ET-3830
4,HP,HP ENVY 6055e


<font color=#61BEB0>**Lower Casing:**</font>

In [491]:
# Lower casing (Review Content)
df_raw['Review Content'] = df_raw['Review Content'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_raw['Review Content'].head()

0               🖨 was easy to install and works great.
1    🔆slower print speed than what i’m used to (old...
2                                              𝙻𝚘𝚟𝚎 𝚒𝚝
3    yup, this printer is a slam dunk. :)what an up...
4    you would think something as simple as reconne...
Name: Review Content, dtype: object

In [492]:
# Lower casing (Review title)
df_raw['Review title'] = df_raw['Review title'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_raw['Review title'].head()

0                           works great
1               spunky mid size printer
2                          𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜
3    shaq knows what he's talking about
4                     not user friendly
Name: Review title, dtype: object

<font color=#61BEB0>**Removing Punctuation:**</font>

In [493]:
# Removing punctuation (Review Content)
df_raw['Review Content'] = df_raw['Review Content'].str.replace('[^\w\s]','')
df_raw['Review Content'].head()

C:\Users\peace\AppData\Local\Temp\ipykernel_14052\3496301409.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_raw['Review Content'] = df_raw['Review Content'].str.replace('[^\w\s]','')


0                  was easy to install and works great
1    slower print speed than what im used to old mo...
2                                              𝙻𝚘𝚟𝚎 𝚒𝚝
3    yup this printer is a slam dunk what an upgrad...
4    you would think something as simple as reconne...
Name: Review Content, dtype: object

In [494]:
# Removing punctuation (Review title)
df_raw['Review title'] = df_raw['Review title'].str.replace('[^\w\s]','')
df_raw['Review title'].head()

C:\Users\peace\AppData\Local\Temp\ipykernel_14052\3689238767.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_raw['Review title'] = df_raw['Review title'].str.replace('[^\w\s]','')


0                          works great
1              spunky mid size printer
2                         𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜
3    shaq knows what hes talking about
4                    not user friendly
Name: Review title, dtype: object

<font color=#61BEB0>**Removal of Stop Words:**</font>

In [495]:
# Removal of stop words (Review Content)
stop = stopwords.words('english')
df_raw['Review Content'] = df_raw['Review Content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df_raw['Review Content'].head()

0                             easy install works great
1    slower print speed im used old modelhpojpro 86...
2                                              𝙻𝚘𝚟𝚎 𝚒𝚝
3    yup printer slam dunk upgrade paid little fron...
4    would think something simple reconnecting wifi...
Name: Review Content, dtype: object

In [496]:
# Removal of stop words (Review title)
stop = stopwords.words('english')
df_raw['Review title'] = df_raw['Review title'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df_raw['Review title'].head()

0                works great
1    spunky mid size printer
2               𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜
3     shaq knows hes talking
4              user friendly
Name: Review title, dtype: object

<font color=#61BEB0>**Removal of Common Words:**</font>

In [497]:
# Removal of common words (Review Content)
freq = pd.Series(' '.join(df_raw['Review Content']).split()).value_counts()[:10]
freq

printer     16548
ink          7322
print        7003
hp           5850
one          3392
use          3308
paper        3275
printing     3023
get          2998
set          2783
dtype: int64

In [498]:
freq = list(freq.index)
df_raw['Review Content'] = df_raw['Review Content'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df_raw['Review Content'].head()

0                             easy install works great
1    slower speed im used old modelhpojpro 8600 new...
2                                              𝙻𝚘𝚟𝚎 𝚒𝚝
3    yup slam dunk upgrade paid little front equiva...
4    would think something simple reconnecting wifi...
Name: Review Content, dtype: object

In [499]:
# Removal of common words (Review title)
freq = pd.Series(' '.join(df_raw['Review title']).split()).value_counts()[:10]
freq

printer    2527
great       991
good        789
ink         710
easy        572
hp          531
print       436
set         430
buy         425
quality     421
dtype: int64

In [500]:
freq = list(freq.index)
df_raw['Review title'] = df_raw['Review title'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df_raw['Review title'].head()

0                     works
1           spunky mid size
2              𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜
3    shaq knows hes talking
4             user friendly
Name: Review title, dtype: object

<font color=#61BEB0>**Removal of Rare Words:**</font>

In [501]:
# Removal of rare words (Review Content)
freq = pd.Series(' '.join(df_raw['Review Content']).split()).value_counts()[-10:]
freq

unit4               1
flawlessly6         1
it7                 1
scanner8            1
only9               1
colornot            1
seconds3            1
breakcrackalso      1
capacityqualityi    1
obsessive           1
dtype: int64

In [502]:
freq = list(freq.index)
df_raw['Review Content'] = df_raw['Review Content'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df_raw['Review Content'].head()

0                             easy install works great
1    slower speed im used old modelhpojpro 8600 new...
2                                              𝙻𝚘𝚟𝚎 𝚒𝚝
3    yup slam dunk upgrade paid little front equiva...
4    would think something simple reconnecting wifi...
Name: Review Content, dtype: object

In [503]:
# Removal of rare words (Review title)
freq = pd.Series(' '.join(df_raw['Review title']).split()).value_counts()[-10:]
freq

screaming    1
utter        1
prnter       1
obviously    1
vender       1
donkey       1
plenty       1
illegal      1
sense        1
roto         1
dtype: int64

In [504]:
freq = list(freq.index)
df_raw['Review title'] = df_raw['Review title'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df_raw['Review title'].head()

0                     works
1           spunky mid size
2              𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜
3    shaq knows hes talking
4             user friendly
Name: Review title, dtype: object

<font color=#61BEB0>**Spelling Correction - need help:**</font>

In [518]:
# Spelling correction (Review Content)


In [519]:
# Spelling correction (Review title)


<font color=#61BEB0>**Stemming:**</font>

In [507]:
# Stemming (Review Content)
st = PorterStemmer()
df_raw['Review Content'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

0                                   easi instal work great
1        slower speed im use old modelhpojpro 8600 new ...
2                                                  𝙻𝚘𝚟𝚎 𝚒𝚝
3        yup slam dunk upgrad paid littl front equival ...
4        would think someth simpl reconnect wifi would ...
                               ...                        
10204                                                  nan
10205                                                  nan
10206                                                  nan
10207                                                  nan
10208                                                  nan
Name: Review Content, Length: 10209, dtype: object

In [508]:
# Stemming (Review title)
st = PorterStemmer()
df_raw['Review title'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

0                         work
1              spunki mid size
2                 𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜
3            shaq know he talk
4                user friendli
                 ...          
10204           quick deliveri
10205                   awesom
10206                     work
10207        el artículo llegó
10208    new work like one use
Name: Review title, Length: 10209, dtype: object

<font color=#61BEB0>**Lemmatization:**</font>

In [509]:
# Lemmatization (Review Content)
df_raw['Review Content'] = df_raw['Review Content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df_raw['Review Content'].head()

0                              easy install work great
1    slower speed im used old modelhpojpro 8600 new...
2                                              𝙻𝚘𝚟𝚎 𝚒𝚝
3    yup slam dunk upgrade paid little front equiva...
4    would think something simple reconnecting wifi...
Name: Review Content, dtype: object

In [510]:
# Lemmatization (Review title)
df_raw['Review title'] = df_raw['Review title'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df_raw['Review title'].head()

0                    work
1         spunky mid size
2            𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜
3    shaq know he talking
4           user friendly
Name: Review title, dtype: object

In [511]:
# Export to csv for use
df_raw.to_excel('amazon_review_processed.xlsx', index=False)

In [512]:
df_processed = pd.read_excel("amazon_review_processed.xlsx")
df_processed.head(5)

,Review Model,Retailer,Review date,Review name,Review rating,Review title,Review Content,Verified Purchase or not,People_find_helpful,vine or not,...,overall rating,content_word_count,title_word_count,content_char_count,title_char_count,content_avg_word,title_avg_word,content_stopwords,title_stopwords,Brand
0,Canon Pixma TS6420a,Amazon,2023-08-10,Ernest Birkholz,5,work,easy install work great,Verified Purchase,0.0,Not VINE VOICE,...,4.2,8,2,38,11,3.875000,5.000000,3,0,Canon
1,HP OfficeJet Pro 9015e,Amazon,2022-06-04,mattey,3,spunky mid size,slower speed im used old modelhpojpro 8600 new...,Verified Purchase,0.0,Not VINE VOICE,...,4.2,79,4,460,23,4.835443,5.000000,19,0,HP
2,Canon PIXMA MG3620,Amazon,2023-03-15,Maria D,4,𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜,𝙻𝚘𝚟𝚎 𝚒𝚝,Verified Purchase,0.0,Not VINE VOICE,...,4.2,2,2,7,12,3.000000,5.500000,0,0,Canon
3,Epson - ET-3830,Amazon,2022-11-17,Ryan H,5,shaq know he talking,yup slam dunk upgrade paid little front equiva...,Verified Purchase,12.0,Not VINE VOICE,...,4.2,84,6,469,34,4.595238,4.833333,31,2,Epson
4,HP ENVY 6055e,Amazon,2022-04-09,Sam,3,user friendly,would think something simple reconnecting wifi...,Verified Purchase,0.0,Not VINE VOICE,...,4.2,160,3,861,17,4.387500,5.000000,71,0,HP
